# Webscraping!

In [1]:
import requests
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
# Athlete ID range (I checked manually)
start_id = 1
end_id = 16460

/Users/maxvazquez/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Athlete Information (birthday, country, height, etc)

In [11]:
# Function to fetch athlete data
def fetch_athlete_data(athlete_id):
    headers = {
        'X-Csrf-Token': 'QsiFWuxEY1S9h_-dQgRA_7S5w9uvvmXsjq56QbTPw4i_g_XR68rMCBFFhW6HngBRtHskfN5yjX8GQmawqs8BlQ',
        'Referer': 'https://ifsc.results.info',
        'Cookie': 'session_id=_verticallife_resultservice_session=6RHN3xZrXnftTiScNfSHg7BVvuebLzGAmC9P5vIpzdySn2vG7VwQpjSZRDHug%2BPKCWlkt831HjLvHsPoVKrzTGsPVR6mqSOtjHB%2Bwht%2Bj39KxYO%2FJlaU6zmh8VhNFEl9bXHiOlPGk8AxnZqiBSYKTxJFCqh34nqdurXfFDcsRnbEtYCixcOdx%2F32E4zYGLVw7DSXXIKOVTUivS43UJZq5zDWPctX95UWm%2FD7%2B6UYT2s0B%2B3XJVPgjMWCMR%2FVZs%2FQC45Gjm4uCpHHe8Yt73nM3J%2Br43V1HuHGSvRpRczrJ4QdovlJHDEpg4rjUA%3D%3D--xlV%2BnBWvX%2BFwNcXI--24wJLAwE%2F8YbvCRvH9nMWQ%3D%3D',  # If session cookies are required
    }
    url = f"https://ifsc.results.info/api/v1/athletes/{athlete_id}"
    try:
        response = requests.get(url, headers=headers, timeout=120)
        if response.status_code == 200:
            athlete_data = response.json()
            return {
                'athlete_id': athlete_data['id'],
                'firstname': athlete_data['firstname'],
                'lastname': athlete_data['lastname'],
                'age': athlete_data['age'],
                'gender': athlete_data['gender'],
                'country': athlete_data['country'],
                'height': athlete_data['height'],
                'arm_span': athlete_data['arm_span'],
                'paraclimbing_sport_class': athlete_data['paraclimbing_sport_class'],
                'birthday': athlete_data['birthday'],
            }
        else:
            return None  # Handle non-200 responses gracefully
    except Exception as e:
        print(f"Error fetching data for athlete ID {athlete_id}: {e}")
        return e

In [14]:
def retry_failed_athlete_info(failed_ids, max_retries=2, delay=2):
    retry_results = []
    for retry_count in range(max_retries):
        print(f"Retry attempt {retry_count + 1} for {len(failed_ids)} failed athlete IDs")
        retry_futures = []
        with ThreadPoolExecutor(max_workers=20) as executor:
            retry_futures = {executor.submit(fetch_athlete_data, athlete_id): athlete_id for athlete_id in failed_ids}
            failed_ids = []  # Reset failed_ids list for next retry
        
            for future in as_completed(retry_futures):
                athlete_data = future.result()
                athlete_id = retry_futures[future]

                if isinstance(athlete_data, Exception): # If an error occured during the fetch
                    failed_ids.append(athlete_id)
                elif athlete_data:
                    retry_results.append(athlete_data)
        
        if not failed_ids:
            break  # Exit loop if no more failed IDs
        
        time.sleep(delay)  # Wait between retries to avoid overloading the server
    
    if failed_ids:
        print(f"Final failed athlete IDs after {max_retries} retries: {failed_ids}")
    
    return retry_results, failed_ids


# Function to scrape athlete data in parallel
def scrape_athletes_parallel(start_id, end_id, max_workers=25):
    athletes_info = []
    failed_ids = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit tasks for each athlete ID
        futures = {executor.submit(fetch_athlete_data, athlete_id): athlete_id for athlete_id in range(start_id, end_id + 1)}
        
        for future in as_completed(futures):
            athlete_data = future.result()
            athlete_id = futures[future]

            if isinstance(athlete_data, Exception):  # If an error occured during the fetch
                failed_ids.append(athlete_id)
            elif athlete_data:
                athletes_info.append(athlete_data)
    

    # Retry for failed athlete IDs
    if failed_ids:
        retry_results, failed_ids = retry_failed_athlete_info(failed_ids)
        athletes_info.extend(retry_results)  # Add successful retries

    return athletes_info, failed_ids



# Scrape athlete data from the range and create a DataFrame
athletes_info_list, failed_ids = scrape_athletes_parallel(start_id, end_id)
athletes_info_df = pd.DataFrame(athletes_info_list)

athletes_info_df.to_csv('athlete_information.csv', index=False)

print(f"Scraped {len(athletes_info_df)} athletes")
if failed_ids:
    print(f"Failed to fetch data for {len(failed_ids)} athlete IDs after retries: {failed_ids}")

Error fetching data for athlete ID 6518: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))
Error fetching data for athlete ID 6520: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))
Error fetching data for athlete ID 6516: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))
Error fetching data for athlete ID 6517: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))
Error fetching data for athlete ID 6519: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))
Error fetching data for athlete ID 6521: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))
Error fetching data for athlete ID 6510: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))
Error fetching data for athlete ID 6512: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))
Error fetching data for athlete ID 6523: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))
Error fetching data for athlete ID 6522: ('Con

## Athlete Results

In [5]:
def fetch_athlete_results(athlete_id):
    headers = {
        'X-Csrf-Token': 'QsiFWuxEY1S9h_-dQgRA_7S5w9uvvmXsjq56QbTPw4i_g_XR68rMCBFFhW6HngBRtHskfN5yjX8GQmawqs8BlQ',
        'Referer': 'https://ifsc.results.info',
        'Cookie': 'session_id=_verticallife_resultservice_session=6RHN3xZrXnftTiScNfSHg7BVvuebLzGAmC9P5vIpzdySn2vG7VwQpjSZRDHug%2BPKCWlkt831HjLvHsPoVKrzTGsPVR6mqSOtjHB%2Bwht%2Bj39KxYO%2FJlaU6zmh8VhNFEl9bXHiOlPGk8AxnZqiBSYKTxJFCqh34nqdurXfFDcsRnbEtYCixcOdx%2F32E4zYGLVw7DSXXIKOVTUivS43UJZq5zDWPctX95UWm%2FD7%2B6UYT2s0B%2B3XJVPgjMWCMR%2FVZs%2FQC45Gjm4uCpHHe8Yt73nM3J%2Br43V1HuHGSvRpRczrJ4QdovlJHDEpg4rjUA%3D%3D--xlV%2BnBWvX%2BFwNcXI--24wJLAwE%2F8YbvCRvH9nMWQ%3D%3D',  # If session cookies are required
    }
    url = f"https://ifsc.results.info/api/v1/athletes/{athlete_id}"
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            athlete_data = response.json()
            # Extract the athlete's results from the 'all_results' field
            results = []
            for result in athlete_data.get('all_results', []):
                results.append({
                    'athlete_id': athlete_id,
                    'rank': result['rank'],
                    'discipline': result['discipline'],
                    'season': result['season'],
                    'date': result['date'],
                    'event_id': result['event_id'],
                    'event_location': result['event_location'],
                    'd_cat': result['d_cat'],
                })
            return results
        else:
            return None  # Handle non-200 responses gracefully
    except Exception as e:
        print(f"Error fetching results for athlete ID {athlete_id}: {e}")
        return e


In [7]:
def retry_failed_athletes(failed_ids, max_retries=3, delay=2):
    retry_results = []
    for retry_count in range(max_retries):
        print(f"Retry attempt {retry_count + 1} for {len(failed_ids)} failed athlete IDs")
        retry_futures = []
        with ThreadPoolExecutor(max_workers=20) as executor:
            retry_futures = {executor.submit(fetch_athlete_results, athlete_id): athlete_id for athlete_id in failed_ids}
            failed_ids = []  # Reset failed_ids list for next retry
        
            for future in as_completed(retry_futures):
                results = future.result()
                athlete_id = retry_futures[future]

                if isinstance(results, Exception): # If an error occured during the fetch
                    failed_ids.append(athlete_id)
                elif results:
                    retry_results.extend(results)
        
        if not failed_ids:
            break  # Exit loop if no more failed IDs
        
        time.sleep(delay)  # Wait between retries to avoid overloading the server
    
    if failed_ids:
        print(f"Final failed athlete IDs after {max_retries} retries: {failed_ids}")
    
    return retry_results, failed_ids



def scrape_athlete_results_parallel(start_id, end_id, max_workers=60):
    athlete_results = []
    failed_ids = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_athlete_results, athlete_id): athlete_id for athlete_id in range(start_id, end_id + 1)}
        
        for future in as_completed(futures):
            results = future.result()
            athlete_id = futures[future]
            
            if isinstance(results, Exception):  # If an error occured during the fetch
                failed_ids.append(athlete_id)
            elif results:
                athlete_results.extend(results)

                if athlete_id % 1000 == 0:
                    print(f"Processed athlete ID {athlete_id}")
                
    
    # Retry for failed athlete IDs
    if failed_ids:
        retry_results, failed_ids = retry_failed_athletes(failed_ids)
        athlete_results.extend(retry_results)  # Add successful retries
    
    return athlete_results, failed_ids


athlete_results_list, failed_ids = scrape_athlete_results_parallel(start_id, end_id)
athlete_results_df = pd.DataFrame(athlete_results_list)

athlete_results_df.to_csv('athlete_results.csv', index=False)

print(f"Scraped results for {len(athlete_results_df)} athlete events")
if failed_ids:
    print(f"Failed to fetch data for {len(failed_ids)} athlete IDs after retries: {failed_ids}")

Processed athlete ID 1000
Processed athlete ID 2000
Error fetching results for athlete ID 936: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))
Processed athlete ID 5000
Processed athlete ID 6000
Processed athlete ID 7000
Processed athlete ID 8000
Processed athlete ID 9000
Processed athlete ID 10000
Processed athlete ID 11000
Processed athlete ID 12000
Processed athlete ID 14000
Processed athlete ID 15000
Error fetching results for athlete ID 16013: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))
Retry attempt 1 for 2 failed athlete IDs
Scraped results for 135089 athlete events
